# **Importações e uploads necessários**

In [ ]:
import json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_json('/content/drive/MyDrive/final_dataframe.json')
df_by_id_chat = df.set_index('_id_chat')
df_by_id_chat

In [ ]:
id_dialogos = df['_id_chat'].unique()
id_dialogos.size

1388

# **Extração das ações através das respostas do chatbot**

In [ ]:
actions_map = {
        'error-treatment-v1': 'Não consegui entender o que você quis dizer',
        'error-treatment-v2': 'Não consegui entender.',
        'error-treatment-v3': 'Desculpe, ainda estou aprendendo. Escreva o que você precisa, por favor.',
        'error-treatment-voice-message': 'Desculpe, ainda não consigo entender conversas por voz.',
        'show-welcome-message': 'Olá! Bem-vindo(a) ao serviço de atendimento online da Secretaria da Saúde do Estado do Ceará!',
        'start-dialog': 'Do que você precisa agora? (Digite o número da opção desejada) 1.  Atendimento online sobre o novo Coronavírus (COVID-19)  2.  Informações sobre o novo Coronavírus (COVID-19)  3.  Cuidados com a Saúde Mental 4. Outros',
        'error-treatment-start-dialog-v1': 'Desculpe, não entendi sua dúvida. Do que você precisa agora? (Digite o número da opção desejada) 1.  Atendimento online sobre o novo Coronavírus (COVID-19)  2.  Informações sobre o novo Coronavírus (COVID-19)  3.  Orientações sobre cuidados com a Saúde Mental 4. Outros',
        'error-treatment-start-dialog-v2': 'Desculpe, não entendi sua dúvida. Do que você precisa agora? (Digite o número da opção desejada)   1.  Atendimento online sobre o novo Coronavírus (COVID-19)  2.  Informações sobre o novo Coronavírus (COVID-19)  3.  Cuidados com a Saúde Mental 4. Outros',
        'start-online-service-v1': 'As informações compartilhadas aqui são protegidas e ajudam o Governo do Estado a combater o novo Coronavírus (COVID-19). ',
        'start-online-service-v2': ' É muito importante responder todas as perguntas e completar o atendimento. Ao final, se for necessário, uma equipe de saúde estará pronta para conversar e cuidar de você.',
        'start-ask-patient-questions': 'Entendi, obrigado! Agora, preciso saber um pouco mais sobre você.',
        'ask-patient-symptoms-v1': 'Você está com algum desses sintomas?   1. Falta de ar  2. Mal-estar intenso  3. Tosse incontrolável  4. Alargamento das narinas durante a respiração (para crianças)  5. Febre acima de 37.8°C persistente por mais de 48h  6. Confusão mental   7. Nenhum dos sintomas acima   Digite os números dos sintomas separados por vírgula (exemplo: 2, 5) ou digite 7 se não estiver com os sintomas descritos acima.',
        'ask-patient-symptoms-v2': 'Você está com algum desses sintomas?  Selecione uma ou mais das opções abaixo.',
        'ask-patient-symptoms-v3': 'Você está com algum desses outros sintomas?  1. Febre  2. Tosse  3. Nariz escorrendo  4. Dor de cabeça  5. Dor de garganta  6. Dores pelo corpo  7. Diarreia  8. Nenhum dos sintomas acima   Digite os números dos sintomas separados por vírgula (exemplo: 2, 5, 7) ou digite 8 se não estiver com os sintomas descritos acima.',
        'ask-patient-symptoms-v4': 'Você está com algum desses outros sintomas? Selecione uma ou mais das opções abaixo.',
        'ask-patient-symptoms-v5': 'Quais são os sintomas? Pode digitar os sintomas *separado por vírgula* (exemplo: Febre alta, tosse incontrovável e mal estar) ou digite *não* se não estiver.',
        'show-info-about-not-feeling-symptoms': 'você nos disse que não está sentindo nenhum dos sintomas!  Continuaremos em contato nos próximos dias para acompanhar qualquer mudança. Quando você precisar falar de novo com a gente, basta voltar aqui e escrever "orientação" para iniciar nova conversa com um profissional de saúde.',
        'ask-patient-how-many-days-symptoms': 'Há quantos dias os sintomas começaram? Responda apenas com o número de dias (exemplo: 2).',
        'error-treatment-patient-how-many-days-symptoms': 'Por favor, digite uma quantidade de dias válida.',
        'call-diagnostic-system': 'O sistema está verificando se você está com suspeita de covid.',
        'ask-share-location-v1': 'Vamos direcioná-lo(a) para nossa equipe de saúde. Clique no botão "Compartilhar minha localização" para compartilhar onde você está. Ao compartilhar sua localização, você ajuda o Governo a te proteger e proteger os outros. Caso não deseje compartilhar, clique no botão "Não quero compartilhar minha localização".',
        'ask-share-location-v2': 'Vamos direcioná-lo(a) para nossa equipe de saúde. Clique no botão ""Compartilhar minha localização"" para compartilhar onde você está. Ao compartilhar sua localização, você ajuda o Governo a te proteger e proteger os outros. Caso não deseje compartilhar, clique no botão ""Não quero compartilhar minha localização"".',
        'ask-share-location-v3': 'Vamos direcioná-lo(a) para nossa equipe de saúde. Clique no  ou no , selecione "Localização" e depois "Localização fixa". Ao compartilhar sua localização, você ajuda o Governo a te proteger e proteger os outros. Caso não deseje compartilhar, digite "Não".',
        'ask-share-location-v4': 'Clique no  ou no , selecione "Localização" e depois "Localização fixa" e compartilhe onde você está. Ao fazer isso, você ajuda o Governo a cuidar melhor de você. Caso não deseje compartilhar, digite "Não".',
        'ask-share-location-v5': 'Precisamos da sua localização para controlar a contaminação pelo vírus.  Clique no botão "Compartilhar minha localização" para compartilhar onde você está. Caso não deseje compartilhar, clique no botão "Não quero compartilhar minha localização".',
        'ask-share-location-v6': 'Precisamos da sua localização para controlar a contaminação pelo vírus.  Clique no  ou no , selecione "Localização" e depois "Localização fixa". Ao compartilhar sua localização, você ajuda o Governo a te proteger e proteger os outros. Caso não deseje compartilhar, digite "Não".',
        'ask-share-location-v7': 'Clique no botão "Compartilhar minha localização" e compartilhe onde você está. Ao fazer isso, você ajuda o Governo a cuidar melhor de você. Caso não deseje compartilhar, clique no botão "Não quero compartilhar minha localização".',
        'show-info-about-share-location': 'Localização compartilhada com sucesso! ',
        'error-treatment-share-location': 'O compartilhamento da localização falhou.  Vamos tentar de outra forma.',
        'ask-postal-code': 'Informe o CEP da sua localidade, por favor. Exemplo: 57000-000 ou 57000000. Se não souber seu CEP, digite "Não".',
        'error-treatment-postal-code': 'CEP inválido! Por favor, me informe um CEP válido com o formato abaixo: Exemplo: 57000-000 ou 57000000',
        'ask-patient-info-state': 'Qual o seu estado? Exemplo: Ceará ou CE',
        'ask-patient-info-city': 'Qual a sua cidade?',
        'error-treatment-patient-info-city': 'Essa cidade não existe no Estado. Por favor, diga uma cidade existente.',
        'ask-patient-info-cpf': 'Você pode me informar seu CPF? Não precisa digitar os pontos ou traços ',
        'error-treatment-patient-info-cpf': 'CPF inválido! Por favor, me informe um CPF válido com o formato: 000.000.000-00 ou 00000000000. (se não se sentir a vontade, digite *Não*)',
        'ask-patient-info-name': 'Qual seu nome completo?',
        'ask-patient-info-birthday': 'Qual a sua data de nascimento? Por favor, insira uma data no formato dd/mm/aaaa.',
        'error-treatment-patient-info-birthday': 'Data inválida! Por favor, digite uma data válida no formato dd/mm/aaaa.',
        'ask-patient-comorbidities-v1': 'Você tem algum desses problemas de saúde? 1. Diabetes 2. Pressão alta 3. Problemas nos rins 4. Problemas respiratórios 5. Problemas cardíacos 6. Deficiências imunológicas: portador de HIV, em tratamento para algum câncer, portador de doenças autoimunes (como lúpus, síndrome antifosfolípide, ...), realizou transplante ou usa corticoides 7. Nenhum dos problemas acima Digite os números dos sintomas separados por vírgula (exemplo: 2, 5) ou digite 7 se não estiver com os sintomas descritos acima.',
        'ask-patient-comorbidities-v2': 'Você tem algum desses problemas de saúde? Selecione uma ou mais das opções abaixo. *Considere deficiência imunológica: portador de HIV, em tratamento para algum câncer, portador de doenças autoimunes (como lúpus, síndrome antifosfolípide, ...), realizou transplante ou usa corticoides',
        'ask-patient-comorbidities-v3': 'Você tem algum desses problemas de saúde?    1. Diabetes  2. Pressão alta  3. Problemas nos rins  4. Problemas respiratórios  5. Problemas cardíacos  6. Deficiências imunológicas: portador de HIV, em tratamento para algum câncer, portador de doenças autoimunes (como lúpus, síndrome antifosfolípide, ...), realizou transplante ou usa corticoides  7. Nenhum dos problemas acima   Digite os números dos problemas separados por vírgula (exemplo: 2, 5) ou digite 7 se não tiver os problemas de saúde descritos acima.',
        'ask-patient-info-gender': 'Você é homem, mulher ou prefere não declarar?',
        'error-treatment-patient-info-gender': 'Desculpa não entendi, poderia porfavor informe seu sexo.',
        'ask-patient-info-pregnant-woman': 'Você está grávida?',
        'ask-patient-info-phone-number': 'me informa o número do seu celular, por favor? Não esquece do DDD. Exemplo: (82) 90000-0000 ou 82912345678',
        'error-treatment-patient-info-phone-number': 'Por favor, me informe um número de telefone celular válido. Exemplo: (82) 90000-0000 ou 82912345678',
        'show-info-about-mental-health-v1': 'você gostaria de tirar alguma dúvida sobre os temas a seguir? (Basta digitar o número, um por vez, para acessar a resposta) 1) Como cuidar da minha saúde mental durante a pandemia?  2) Como lidar com as crianças nesse período de isolamento? 3) Como lidar com os idosos nesse período? 4) Como apoiar alguém que está muito ansioso com o novo Coronavírus (COVID-19)? 5) Onde encontro serviços de saúde mental durante a pandemia? 6) Fale com a Equipe de Saúde Mental online 7) Para encerrar seu atendimento',
        'show-info-about-mental-health-v2': 'Agora que sabemos que você está se sentindo bem fisicamente, gostaríamos de saber como você está se sentindo emocionalmente. Você precisa de cuidados com sua saúde mental? 1. Não, pois estou conseguindo lidar com o que está acontecendo 2. Sim, conversar com a Equipe de Saúde Mental seria bom para mim',
        'show-info-mental-health-option-01-v1': 'Alimente-se bem e se exercite Cultive seus hobbies, pratique atividades artísticas Medite Fortaleça os laços com a família e os amigos Diminua o tempo gasto com notícias e busque fontes confiáveis Proteja-se e ajude o próximo Divulgue notícias positivas e de esperança Evite combater o estresse usando tabaco, álcool ou outras drogas Peça ajuda a alguém de sua confiança, se sentir que está sobrecarregado(a) emocionalmente Pense no que o(a) ajudou a lidar com os problemas no passado e o que pode fazer para se fortalecer.',
        'show-info-mental-health-option-01-v2': 'Alimente-se bem e se exercite  Cultive seus hobbies, pratique atividades artísticas  Medite  Fortaleça os laços com a família e os amigos   Diminua o tempo gasto com notícias e busque fontes confiáveis  Proteja-se e ajude o próximo   Divulgue notícias positivas e de esperança  Evite combater o estresse usando tabaco, álcool ou outras drogas  Peça ajuda a alguém de sua confiança, se sentir que está sobrecarregado(a) emocionalmente  Pense no que o(a) ajudou a lidar com os problemas no passado e o que pode fazer para se fortalecer',
        'show-info-mental-health-option-02': 'É importante e saudável para as crianças:   Conversar honestamente sobre o coronavírus e de acordo com a idade     Ajudá-las a expressar seus sentimentos (medo, tristeza, saudade)  Criar um ambiente seguro, com apoio e rotina familiar  Estimulá-las a brincar, aprender e socializar com outras crianças usando a criatividade',
        'show-info-mental-health-option-04': 'Você apoia uma pessoa quando a escuta e conversa com ela de modo positivo.  Esteja atento(a) para os fatos escutados: são verdadeiros?  Se achar necessário, ajude a pessoa a se informar melhor. Mas quando essa escuta gerar ansiedade em você, permita-se diminuir o contato. Oriente a pessoa a procurar ajuda profissional em nossos canais de comunicação.',
        'show-info-mental-health-option-05': 'Os serviços de saúde mental do Estado do Ceará estão prontos para acolher  e cuidar de você. Os Centros de Atenção Psicossocial (CAPS) estão funcionando, adaptados para o período de pandemia. - Acesse um desses canais: Secretaria da Saúde do Estado (Coordenadoria de Saúde Mental)   3101.2663  copom.saudece@gmail.com  www.saude.ce.gov.br Hospital de Saúde Mental de Messejana Prof. Frota Pinto  3101.4348  www.hsmm.ce.gov.br Secretaria de Saúde de Fortaleza  3452.6940  www.fortaleza.ce.gov.br',
        'show-info-about-covid': 'você gostaria de tirar alguma dúvida sobre os temas a seguir? (digite um número de cada vez para ver a resposta) 1) O que é novo Coronavírus (COVID-19)?  2) Como faço para me proteger?  3) Quais são os principais sintomas?  4) Qual o tratamento?  5) Quais são os riscos?  6) O que fazer quando sentir os sintomas?   7) Mitos e verdades sobre a doença  8) Informações sobre o novo Coronavírus no Estado do Ceará  ou 9) Para encerrar seu atendimento. ',
        'show-info-about-new-corona-virus': 'Coronavírus é uma família de vírus que causam infecções respiratórias. Um novo Coronavírus foi descoberto em dezembro de 2019, na China. Ele causa a doença chamada de COVID-19.',
        'show-info-about-main-symptoms-v1': 'Os principais sintomas da doença do novo Coronavírus são: ✓Falta de ar ✓Febre ✓Tosse seca',
        'show-info-about-main-symptoms-v2': ' ✓ Tosse seca',
        'show-info-about-the-treatments': 'Até o momento, não há vacina nem medicamento para prevenir ou tratar o novo Coronavírus (COVID-19). As pessoas afetadas devem receber cuidados gerais, e aquelas com doenças graves devem procurar atendimento de saúde.',
        'show-info-about-covid-in-ceara-v1': 'Para mais informações, acesse os canais oficiais da Secretaria da Saúde do Estado do Ceará (SESA), do Ministério da Saúde e da Organização Mundial da Saúde (OMS). ',
        'show-info-about-covid-in-ceara-v2': 'Mais informações sobre o novo Coronavírus (COVID-19) podem ser acessadas pelos canais oficiais da Secretaria da Saúde do Estado do Ceará (SESA), do Ministério da Saúde e da Organização Mundial da Saúde (OMS).',
        'show-info-about-covid-in-ceara-v3': 'https://www.saude.ce.gov.br',
        'show-info-about-covid-in-ceara-v4': 'https://www.saude.gov.br',
        'show-info-about-covid-in-ceara-v5': 'https://coronavirus.ceara.gov.br',
        'show-info-about-covid-in-ceara-v6': 'Colabore! Não divulgue conteúdos que não tenham sido produzidos por fontes confiáveis e não espalhe fake news. ',
        'show-info-about-how-to-protect-from-virus': 'Consegui entender que você quer saber mais sobre proteção e prevenção. O que você gostaria de saber? 1. Quais as principais medidas de prevenção? 2. Devo usar máscara? 3. Como deve ser a minha máscara caseira? 4. Posso usar minha máscara caseira mais de uma vez? 5. Como lavar as mãos da maneira certa? 6. Como os profissionais de saúde podem se proteger do novo Coronavírus (COVID-19)?',
        'show-info-about-prevention-measure-v1': '- Beber bastante água e líquidos e se alimentar bem',
        'show-info-about-prevention-measure-v2': ' Se você acha que uma superfície pode estar infectada, limpe-a com um desinfetante e em seguida lave as mãos com água e sabão.',
        'show-info-about-prevention-measure-v3': 'Se tiver febre e tosse, somados à dificuldade de repirar, procure atendimento médico',
        'show-info-about-wear-mask': 'undefined path',
        'show-info-about-homemade-mask-look-like': 'undefined path',
        'show-info-about-homemade-mask-use': '✔ Você pode usar sua máscara caseira mais de uma vez, mas tenha os seguintes cuidados: 1. Lave bem as mãos e tire a máscara sem tocar na parte da frente. 2. Deixe a máscara de molho por meia hora em água + água sanitária. 3. Enxágue e lave com água e sabão. Espere secar para usá-la de novo. ⚠ Atenção! Leve com você reservas de máscara, porque o tempo adequado de uso é de 2 horas.',
        'show-info-about-wash-hands-correctly': 'undefined path',
        'show-info-health-professional-protection-v1': '- óculos de proteção',
        'show-info-health-professional-protection-v2': 'Todos os protocolos oficiais devem ser seguidos e estão na página da Secretaria da Saúde do Estado (SESA).',
        'show-info-health-professional-protection-v3': 'Acesse e consulte https://bit.ly/3bfFF2G',
        'show-info-about-the-riscs': 'Consegui entender que você quer saber mais sobre riscos. O que você gostaria de saber? 1. Como essa doença se apresenta? 2. Como essa doença se espalha? 3. Essa doença é transmitida pelo ar? 4. Devo me preocupar com essa doença? 5. Quem é grupo de risco?',
        'show-info-about-disease-present-itself': '- Os sintomas demoram de 1 a 14 dias para aparecer. - A maioria das pessoas se recupera sem precisar de tratamento especial. - A doença pode se agravar em idosos ou naqueles que já tenham outros problemas de saúde (pressão alta, diabetes ou problemas no coração). - As crianças geralmente têm sintomas leves ou não apresentam sintomas.',
        'show-info-about-disease-spread': '- Superfícies e objetos contaminados (se você tocá-los e logo em seguida levar as mãos à boca, aos olhos ou ao nariz).',
        'show-info-about-disease-transmitted-through-the-air-v1': 'A doença não é transmitida pelo ar. Até agora, estudos indicam que o vírus é passado principalmente pelo contato com gotículas respiratórias.',
        'show-info-about-disease-transmitted-through-the-air-v2': 'A doença *não* é transmitida pelo ar. Até agora, estudos indicam que o vírus é passado principalmente pelo contato com gotículas respiratórias.',
        'show-info-about-worried-about-this-disease': 'undefined path',
        'show-info-about-the-risk-group-v1': 'Idosos e pessoas com outros problemas de saúde (diabetes, pressão alta e problemas de coração, por exemplo) são grupo de risco.',
        'show-info-about-the-risk-group-v2': '- As crianças geralmente têm sintomas leves ou não apresentam sintomas.',
        'show-info-about-when-feel-the-symptoms-v1': 'Consegui entender que você quer saber mais sobre sintomas. O que você gostaria de saber? 1. Não me sinto bem. Devo procurar um médico? 2. Onde buscar atendimento? 3. Como é feito o diagnóstico do novo Coronavírus (COVID-19)? 4. Para quem o exame é indicado? 5. O que é isolamento domiciliar? 6. Quando é preciso fazer isolamento domiciliar? 7. O que é o distanciamento social? 8. Todo caso de COVID-19 precisa ser internado no hospital?',
        'show-info-about-when-feel-the-symptoms-v2': 'Atenção! A febre, isoladamente, mesmo alta, em pessoa sem fator de risco não deve ser considerada sinal de gravidade, se não vier acompanhada desses outros sintomas.',
        'show-info-about-should-see-the-doctor': 'Se você tem sintomas de novo Coronavírus (COVID-19), continue este atendimento virtual para que nossos profissionais possam avaliá-lo(a) e orientá-lo(a). Só procure hospitais e postos de saúde se estiver com sintomas graves.',
        'show-info-about-where-to-get-care': 'https://unidades.saude.ce.gov.br',
        'show-info-about-the-diagnostic-of-the-covid': 'Atualmente, o diagnóstico é feito pela técnica swab, que é o uso de um bastão, parecido com um cotonete, para coletar materiais respiratórios (aspiração de vias aéreas ou coleta de secreções da boca e do nariz).',
        'show-info-about-who-is-the-exam-for-v1': 'O exame é indicado para: ✓ Pessoas com sintomas classificados como Síndrome Respiratória Aguda Grave (SRAG)*✓ Profissionais da saúde com sintomas ✓ Pessoas vulneráveis com síndrome gripal (a partir de 60 anos e jovens com diabetes mellitus, pressão alta, problemas cardíacos, doença pulmonar crônica, câncer e gravidez de risco) * Síndrome Respiratória Aguda Grave (SRAG): febre acompanhada de tosse ou dor de garganta ou falta de ar ou nariz escorrendo.',
        'show-info-about-who-is-the-exam-for-v2': '✓ Profissionais da saúde com sintomas',
        'show-info-about-who-is-the-exam-for-v3': '* Síndrome Respiratória Aguda Grave (SRAG): febre acompanhada de tosse ou dor de garganta ou falta de ar ou nariz escorrendo.',
        'show-info-about-who-is-the-exam-for-v4': '* Síndrome Gripal (SG):  febre acompanhada de tosse ou dor de garganta ou falta de ar ou nariz escorrendo. Em criança com menos de 2 anos: febre de início súbito + sintomas respiratórios (tosse, coriza e obstrução nasal).',
        'show-info-about-home-isolation': 'undefined path',
        'show-info-about-when-home-isolation-is-necessary': 'undefined path',
        'show-info-about-social-distancing': 'undefined path',
        'show-info-about-the-cases-need-hospitalization-v1': 'Nem todo caso de COVID-19 precisa ser internado. Somente pacientes classificados como Síndrome Gripal (SG)* e sinais de alarme (desconforto respiratório, cansaço, respiração rápida) ou Síndrome Respiratória Aguda Grave (SRAG)**. * Síndrome Gripal (SG):  febre acompanhada de tosse ou dor de garganta ou falta de ar ou nariz escorrendo. Em criança com menos de 2 anos: febre de início súbito + sintomas respiratórios (tosse, coriza e obstrução nasal). ** Síndrome Respiratória Aguda Grave (SRAG): indivíduo com febre acompanhada de tosse ou dor de garganta e que apresenta falta de ar.',
        'show-info-about-the-cases-need-hospitalization-v2': 'Nem todo caso de COVID-19 precisa ser internado. Somente pacientes classificados como Síndrome Gripal (SG)* e sinais de alarme (desconforto respiratório, cansaço, respiração rápida) ou Síndrome Respiratória Aguda Grave (SRAG)**.',
        'show-info-about-the-cases-need-hospitalization-v3': '** Síndrome Respiratória Aguda Grave (SRAG): indivíduo com febre acompanhada de tosse ou dor de garganta e que apresenta falta de ar.',
        'show-info-about-myths-and-truths': 'Consegui entender que você quer saber mais sobre mitos. O que você gostaria de saber? 1. Os animais de estimação podem espalhar o novo Coronavírus? 2. Quais as pessoas mais afetadas pelo novo Coronavírus? 3. Os antibióticos são eficazes na prevenção e no tratamento do novo Coronavírus? 4. Posso tomar algum remédio para prevenir ou tratar novo Coronavírus? 5. Quanto tempo o novo Coronavírus sobrevive em superfícies? 6. O novo Coronavírus pode matar? 7. Há restrições para comprar mercadorias vindas de países com casos confirmados da doença?',
        'show-info-about-pets-spread-covid-v1': 'Mas sempre lave as mãos com sabão e água corrente depois de pegar nos animais, pois isso protege você contra várias bactérias comuns.',
        'show-info-about-people-affected-by-covid': 'Idosos e pessoas portadoras de doenças graves (pressão alta, asma, diabetes e doenças cardíacas, por exemplo) são as mais afetadas. Mas, atenção, pessoas de todas as idades podem ser infectadas pelo novo Coronavírus (COVID-19).',
        'show-info-antibiotics-effects': 'Antibióticos *não* são eficazes para prevenir e tratar o novo Coronavírus. Antibióticos só funcionam contra bactérias. O novo Coronavírus (COVID-19) é um vírus',
        'show-info-about-medicine-for-covid': 'Ainda não existem remédios específicos recomendados para prevenir ou tratar o novo Coronavírus (COVID-19).',
        'show-info-about-covid-survive-in-surface': 'undefined path',
        'show-info-about-covid-kill': 'undefined path',
        'show-info-about-severity-of-covid': 'Em geral, a doença é leve, mas pode se agravar. Cerca de 1 em cada 5 pessoas infectadas necessita de cuidados hospitalares.',
        'show-info-about-the-restrictions-to-buy': 'Por enquanto, *não* há indícios ou evidências de que seja preciso evitar a importação de produtos.',
        'show-info-about-follow-program-v1': 'Faça parte do nosso Programa de Acompanhamento de Saúde, envie uma mensagem de Whatsapp com a palavra "Acompanhamento" para o número (85) 98439-0647.  Ou clique no link abaixo para ser direcionado para nosso perfil no Whatsapp: ',
        'show-info-about-follow-program-v2': 'https://bit.ly/2KitO8J',
        'show-info-about-follow-program-v3': 'Assim, mandaremos mensagens para saber como você está se sentindo e garantir que está recebendo a orientação certa.',
        'show-info-about-follow-program-v4': 'A Secretaria da Saúde do Estado tem um Programa de Acompanhamento de Saúde para você.  Mandaremos mensagens para saber como você está se sentindo e garantir que está recebendo a orientação certa.',
        'show-info-about-follow-program-v5': 'Pessoal, o governo lançou um serviço de atendimento pelo WhatsApp que pode nos ajudar.  Basta clicar nesse link para ser atendido(a) online por um profissional de saúde!',
        'show-info-about-follow-program-v6': 'Pessoal, o governo lançou um serviço de atendimento de saúde pelo WhatsApp que pode nos ajudar.  Basta clicar nesse link para ser atendido(a) online por uma equipe de saúde!',
        'show-info-about-follow-program-v7': 'https://bit.ly/ceara_plantaocoronavirus',
        'show-info-about-health-avaliation-v1': 'Aqui além de informações e recomendações para prevenção da doença causada pelo novo Coronavírus, você tem a possibilidade de fazer uma autoavaliação sobre seu estado de saúde dentro de casa, sem se expor a situações de grande de risco, como ao comparecer a uma unidade de saúde apresentando sintomas brandos.',
        'show-info-about-health-avaliation-v2': 'Deseja realizar a autoavaliação do novo Coronavírus?',
        'error-treatment-health-avaliation': 'sua autoavaliação foi cancelada porque você não enviou mais mensagens.  Vamos recomeçar?  Digite "atendimento".',
        'show-info-others-v1': 'Este é um canal para te orientar sobre o novo Coronavírus (COVID-19).     Para denunciar descumprimento a determinações do Governo no enfrentamento ao novo Coronavírus, ligue 190.      Para outras informações, fale com a Central de Atendimento da Ouvidoria pelo 155.',
        'show-info-others-v2': 'Este é um canal exclusivo para orientar a população sobre o novo Coronavírus (COVID-19).  Para denunciar descumprimento a determinações do Governo no enfrentamento ao novo Coronavírus, ligue 190.   Para outras informações, fale com a Central de Atendimento da Ouvidoria pelo 155.',
        'ask-user-grade-v1': 'Certo! Antes de encerrar peço 1 minutinho para que você avalie o meu atendimento (digite um número de 1 a 10), sendo: 1. Odiei.  >>> 10. Amei! ',
        'ask-user-grade-v2': 'Por favor, antes de desconectar, avalie esse serviço. Dê uma nota de 1 a 10, sendo:   1.Ruim  >>> 10. Muito bom! ',
        'error-treatment-user-grade-v1': 'Não entendi . Por favor, me fala a nota que você dá para o meu atendimento como um número de 1 a 10. Ex.: 8',
        'error-treatment-user-grade-v2': 'Não entendi . Por favor, avalie o meu atendimento com um número de 1 a 10. Ex.: 8',
        'ask-user-check-v1': 'Suas dúvidas foram respondidas?',
        'ask-user-check-v2': 'Quer fazer outra pergunta?',
        'ask-user-check-v3': 'Deseja fazer outra pergunta?',
        'ask-user-check-v4': ' você gostaria de tirar alguma dúvida sobre os temas a seguir?',
        'ask-user-check-v5': 'você nos disse que está com sintomas leves que podem ser relacionados ao novo Coronavírus. Queremos orientá-lo(a) e acompanhá-lo(a) da melhor forma.  Você gostaria de ser atendido(a) por nossa Equipe de Saúde e contar se houve alguma mudança desde o nosso último contato? Digite "sim" ou "não".',
        'error-treatment-user-check': 'Por favor, responda a pergunta com *Sim* ou *Não*',
        'user-check': 'Tudo bem.',
        'user-check-mild-symptoms-v1': 'Entendido, Bom. Continuaremos em contato nos próximos dias para saber como você está se sentindo. Se você perceber qualquer mudança, é importante começar um novo atendimento online. Para isso, basta voltar aqui e escrever "orientação".',
        'user-check-mild-symptoms-v2': 'Continuaremos em contato nos próximos dias para saber como você está se sentindo. Se você perceber qualquer mudança, é importante começar um novo atendimento online. Para isso, basta voltar aqui e escrever "orientação".',
        'user-orientation-find-health-service-center': 'toda a nossa equipe de saúde está em atendimento agora.  Como você apresenta sintomas graves, pedimos que procure a unidade de saúde mais próxima.',
        'user-in-wait-health-profitional': 'guarde alguns minutos, estamos aqui para cuidar de você. No momento, toda a equipe de saúde está em atendimento. Mas não se preocupe, logo iremos atendê-lo(a).',
        'user-case-not-in-ceara': 'Este serviço de atendimento online está disponível somente para o Ceará.  *Procure o TeleSUS (pelo 136) ou o atendimento de saúde de sua localidade.*  Desejamos que fique bem logo! Melhoras!',
        'finish-service': 'Seu atendimento foi finalizado.',
        'finish-dialog-final-v1': 'Obrigado pelo seu contato! Juntos venceremos essa luta contra o novo Coronavírus! #FiqueEmCasa    Até logo!',
        'finish-dialog-final-v2': 'Seja um(a) aliado(a) nessa luta, compartilhe esse serviço com pessoas e grupos que podem precisar da nossa ajuda.   Juntos venceremos essa luta contra o novo Coronavírus! #FiqueEmCasa    Até logo!',
        'finish-dialog-final-v3': '*Lembre-se de que você é parte importante no combate ao novo Coronavírus.* Se proteja e ajude a proteger os outros. Siga as orientações a seguir, especialmente enquanto ainda estiver sentindo algum sintoma:   Lave sempre as mãos com água e sabão    Evite tocar nos olhos, no nariz e na boca   Se estiver tossindo ou espirrando, use máscara     Fique em casa',
        'finish-dialog-final-v4': 'Contamos com você nessa luta contra o novo Coronavírus! Compartilhe esse serviço com pessoas e grupos que precisem da nossa ajuda.  #FiqueEmCasa',
        'finish-dialog-final-v5': 'Juntos venceremos essa luta contra o novo Coronavírus! #FiqueEmCasa    Até logo!',
        'finish-dialog-final-v6': 'Seja um(a) aliado(a) nessa luta, compartilhe esse serviço com pessoas e grupos que podem precisar da nossa ajuda. Juntos venceremos essa luta contra o novo Coronavírus!  #FiqueEmCasa',
        'health-agent-takes-control': 'Olá, obrigado pelas informações! Elas são muito importantes para que nós consigamos atender a todos nesse momento difícil. Nossa equipe de saúde está aqui para te ajudar! Como posso te ajudar?',
        'show-info-calling-health-profitional-v1': 'Estamos acionando um profissional de saúde para atender e cuidar de você.  Por favor, complete as informações a seguir para agilizarmos o seu atendimento.',
        'show-info-calling-health-profitional-v2': 'nossa Equipe de Saúde vai atender e orientar você.',
        'show-info-diagnostic-positive-v1': 'De acordo com suas respostas e com o protocolo do Ministério da Saúde, *você pode estar com o novo Coronavírus (COVID-19).*  ',
        'show-info-diagnostic-positive-v2': 'De acordo com suas respostas e com o protocolo do Ministério da Saúde, *você possui sintomas leves que podem estar relacionados ao novo Coronavírus (COVID-19).*  ',
        'show-info-diagnostic-negative': 'De acordo com suas respostas e com o protocolo do Ministério da Saúde, *você não apresenta sintomas da doença do novo Coronavírus (COVID-19).*  Neste momento, você não precisa de um atendimento com os profissionais de saúde. Eles estão focados no acolhimento de casos graves. É muito importante que, ao perceber qualquer mudança, você volte aqui e escreva "atendimento". Isso fará com que você possa iniciar um novo atendimento online. #FiqueEmCasa  e siga as orientações das autoridades competentes.',
        'call-health-agent-v1': 'Obrigado por se cadastrar. Permaneça com a conversa aberta. Em alguns minutos iremos atendê-lo(a). ',
        'call-health-agent-v2': 'Obrigado por se cadastrar. Permaneça com a conversa aberta. Em alguns minutos iremos atendê-lo(a).',
        'call-health-agent-v3': 'Vou te encaminhar para nossa equipe de saúde. ',
        'call-health-agent-v4': 'Vamos direcioná-lo(a) para nossa Equipe de Saúde Mental. ',
        'call-health-agent-v5': 'Continue com a nossa conversa aberta. A Equipe de Saúde Mental vai atendê-lo(a) em alguns minutos. ',
        'call-health-agent-v6': 'Não se preocupe, mantenha a conversa aberta que logo iremos atendê-lo(a)!',
        'call-health-agent-v7': 'Obrigado por responder. Permaneça com a conversa aberta. Iremos atendê-lo(a) em alguns minutos!',
        'show-initial-message-follow-program-v1': 'Queremos estar mais próximos de você através do nosso Programa de Acompanhamento de Saúde. ',
        'show-initial-message-follow-program-v2': 'Estamos de volta com nosso Programa de Acompanhamento de Saúde para estar mais próximos de você. ',
        'error-treatment-follow-program': 'não recebemos sua resposta!  É muito importante sabermos como você está para seguirmos te orientando. Digite "acompanhamento", se você quer continuar no nosso Programa de Acompanhamento!',
        'start-dialog-follow-program-v1': 'Agora, faremos algumas perguntas sobre como você está se sentindo hoje.',
        'start-dialog-follow-program-v2': 'As informações compartilhadas aqui são protegidas e ajudam o Governo do Estado no combate ao novo Coronavírus (COVID-19).'
    }

In [ ]:
def find_actions(chatbot_phrase):

    phrase = chatbot_phrase

    #print('-------', phrase)
    #action = [k for k, v in actions_map.items() if v == phrase]
    action = [key for key, val in actions_map.items() if val in phrase]

    if action:
      action = action[0]


    return action


# **Análise dos diálogos do dataframe**


In [ ]:
# extração das ações a cada interação do chatbot

file = open("/content/drive/MyDrive/dialogs_path.txt","w")

dialogs_with_human = 0
troca_de_turno = False
total_dialogs = 0
not_dataframe = 0

for i in range(id_dialogos.size):
  dialog = df_by_id_chat.loc[id_dialogos[i]]
  if (isinstance(dialog,pd.DataFrame)):
    if dialog.astype(str).isin(['human']).any().any():
      dialogs_with_human += 1
    else:
      line = 'Caminho do diálogo ' + str(i) + '\n'
      file.writelines(line)
      troca_de_turno = False
      total_dialogs += 1
      for index, row in dialog.iterrows():
        if (row['ori'] == 'bot'):
          action = find_actions(row.txt)
          if (not action):
            if (not troca_de_turno):
              file.write('----- troca de turno entre bot e humano -----\n')
              troca_de_turno = True
          else:
            file.writelines(action)
            file.write('\n')
      file.write('\n')
  else:
    not_dataframe += 1

file.close()

print(dialogs_with_human, total_dialogs, not_dataframe)


256 1125 7
